# User Guide

## Quick start

You can install this extension from PyPI using:

```bash
pip install -U jupyterlab-ui-profiler
```

The ui-profiler can be started from Launcher's "Other" section:

![Launcher](images/launcher.png)

### Interface

You can select multiple benchmarks (left) and multiple scenarios (right) at once. Configuration options for benchmarks and scenarios will show up when selected.

![Profiler](images/ui-profiler.png)

In [ ]:
import json
from pathlib import Path
from IPython.display import display, HTML

schema_dir = Path('../../src/schema')

In [ ]:
def format_property(prop: dict, is_required: bool):
    # TODO: default
    detail = prop['type']
    if 'enum' in prop:
        detail = ' | '.join(prop['enum'])
    if 'default' in prop:
        detail += ', default=' + repr(prop['default'])
    out = prop['title'] + ' (' + detail + ')'
    if 'description' in prop:
        out += ': ' + prop['description']
    return out


def show_settings(configurable: str):
    try:
        with open(schema_dir / f'{configurable}.json', 'r') as f:
            settings = json.load(f)
    except Exception as e:
        print(schema_dir)
        print(schema_dir.glob('**/*'))
        raise e
    properties = settings['properties']
    required = settings.get('required', [])
    formatted = [
        format_property(prop, prop_id in required)
        for prop_id, prop in properties.items()
    ]
    formatted = '\n'.join(['<li>' + entry for entry in formatted])
    display(HTML(f'<ul>{formatted}</ul>'))

## Measuring execution time

> Note: The execution time is measured using `performance.now()` method which has reduced resolution in certain browsers/setups.
>
> To enable high-precision timings in Firefox (79+) we send additional HTTP headers: `Cross-Origin-Opener-Policy: same-origin` and `Cross-Origin-Embedder-Policy: require-corp` which may interfere with user experience (for example restricting the download of images from third-party servers). If this is undesirable, disable the server extension with:
> ```
> jupyter server extension disable jupyterlab_ui_profiler
> ```

### Interpretation

In general performance timings are easily skewed towards higher values by external tasks such as garbage collection, activity of browser extensions or CPU utilisation by external processes. We recommend sorting by first quartile as default, with empirically second-best option being inter-quartile mean - but this varies by benchmark and scenario.

For benchmarking of individual actions, box plot visualisation is provided. You can hover over the circles, box lines, and whiskers to get details about the individual datapoints/statistics.

![Boxplot](images/boxplot.png)

The box represents 1st, 2nd (median) and 3rd quartile and whiskers correspond to minimum/maximum or 1.5 IQR away from the 1st/3rd quartile, whichever is smaller.

#### Available settings

In [ ]:
show_settings('benchmark-execution')

### Troubleshooting

If the timings in Firefox are of low resolution even though headers are sent properly, check your privacy settings, especially `privacy.resistFingerprinting`.

## Benchmarking CSS styles

Four benchmarks are provided for CSS styles:
- [Style Rules](style_rules): provides information on CSS rule performance by disabling rules one-by-one, but takes a long time to iterate over every single rule available
- [Style Sheets](style_sheets): disables entire CSS style sheets at a time; it is faster than disabling rules one-by-one, and more efficient when offending styles come from a single extension
- [Style Rule Groups](style_rule_groups): allows to triangulate the offending rules or group of rules by disabling them block-by-block and then calculating an average contribution of each rule based on the runtime of each block
- [Style Rule Usage](style_rule_usage): observes DOM modifications to calculate the frequency of usage for rules deemed relevant to the investigated scenario

### Nomenclature

*Rules* are defined as groups of *properties*:

```html
<!-- sheet start -->
<style>
  /* rule start */
  .jp-MainAreaWidget  /* selector */
  {
    color: red;       /* property */
    border: 1px;      /* property */
  }
  /* rule end */

  /* rule start */
  .jp-Notebook {
    display: block;
  }
  /* rule end */
</style>
<!-- sheet end -->
```

### Limitations

Rules with pseudo-selectors and pseudo-classes in selector (e.g. `:hover`) cannot be invoked programatically, and therefore this ui-profiler cannot rule out performence bottlenecks resulting from such rules.

(style_rules)=
### Style Rules

#### Available settings

In [ ]:
show_settings('benchmark-rule')

(style_sheets)=
### Style Sheets

#### Available settings

In [ ]:
show_settings('benchmark-base')

#### Interpretation

When a stylesheet is highlighted in this analysis, but no individual rule was higlighted in [Style Rules](style_rules) analysis, it may indicate a presence of interaction or synergy between rules.

(style_rule_groups)=
### Style Rule Groups

#### Available settings

In [ ]:
show_settings('benchmark-rule-group')

(style_rule_usage)=
### Style Rule Usage

#### Available settings

In [ ]:
show_settings('benchmark-rule-usage')

## Benchmarking JavaScript

### At a glance

- Benchmarking JavaScript code enables discovery of functions which
  contribute to performance bottlenecks.
- This feature relies on [JS self-profiling API](https://github.com/WICG/js-self-profiling)
  which as of 2022 is only available on Chromium-based browsers.
  - The self-profiling API relies on sampling method to collect stack
    (list of frames) at specified *sampling intervals*.

### Available settings

In [ ]:
show_settings('benchmark-profile')

### Interpretation

### Troubleshooting

To enable self-profiling we add `Document-Policy: js-profiling` header to the settings of jupyter-server (via `jupyterlab_ui_profiler` server extension).

If the profiler is not available, ensure that the server extension is active using:


```
jupyter server extension list
```

if it is active and you have just installed the extension, restart `jupyterlab` to ensure that it is loaded.

If you use a replacement server (e.g. jupyverse) you will need to add the required header manually or switch to the default jupyter-server for the time of profiling.

# Scenarios

TODO: should default scenarios be documented in another document?

# Advanced usage

TODO: possibly in another document?

## Loading from Python